In [2]:
L = [0,7,9,1,5,3]
L.sort()
L

[0, 1, 3, 5, 7, 9]

In [5]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np

In [7]:
L = [0,7,9,1,5,3]
sorted_indices = np.argsort(L)
print(f"sorted_indices = {sorted_indices}")
print(f"L[sorted_indices] = {np.array(L)[sorted_indices]}")

sorted_indices = [0 3 5 4 1 2]
L[sorted_indices] = [0 1 3 5 7 9]


# RNN or Sequential Models in General
Sequential models can deal with input of arbitrary length no problem. However, as one dives deeper into conceiving
a model, one would soon find out that the implementation details requires some design:

01. If we only design a normal sequential model, it will still **suffer from a similar issue** when we wanted to use vanilla NN, i.e. although the input can be now be of arbitrary length, some of the sequential models can only have fixed-length output.
  - In our case, i.e. **the output sequence length equals the input sequence length** [The sorted array's length should of course equal the original array's length], two models are particularly suited: **encoder-decoder** and **seq-to-seq** models.
  - Seq-to-seq models might
02. What kind of output should our sequential model produce?
03. What should the loss function look like?
04. Dataset.

## Dataset
If we take every array into consideration, how many are there?<br>
Well,

- for length-2 arrays, there are `10*9` of them
- for length-3 arrays, there are `10*9*8` of them
- ...
- for length-10 arrays, there are $10!$ of them

Let's calculate their sum using Python.

In [11]:
list(range(10,8))

[]

In [12]:
list(range(10,8,-1))

[10, 9]

In [14]:
import functools

In [19]:
functools.reduce(lambda x, y: x*y, range(10,7,-1))

720

The following cell should give the right number.

In [21]:
sum([functools.reduce(lambda x, y: x*y, range(10,10-length,-1))
     for length in range(2, 10+1)])

9864090

I just wanted to have an estimate of the number of arrays we can have. Now, how should we generate and store our data for training?

Let's first try with brute-force. Store the entire dataset into a dictionary with keys as any array and value as the sorted array.

In [32]:
from itertools import combinations, permutations

In [26]:
S = set(range(0, 9+1))

In [33]:
list(combinations(S, 2))

[(0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (0, 5),
 (0, 6),
 (0, 7),
 (0, 8),
 (0, 9),
 (1, 2),
 (1, 3),
 (1, 4),
 (1, 5),
 (1, 6),
 (1, 7),
 (1, 8),
 (1, 9),
 (2, 3),
 (2, 4),
 (2, 5),
 (2, 6),
 (2, 7),
 (2, 8),
 (2, 9),
 (3, 4),
 (3, 5),
 (3, 6),
 (3, 7),
 (3, 8),
 (3, 9),
 (4, 5),
 (4, 6),
 (4, 7),
 (4, 8),
 (4, 9),
 (5, 6),
 (5, 7),
 (5, 8),
 (5, 9),
 (6, 7),
 (6, 8),
 (6, 9),
 (7, 8),
 (7, 9),
 (8, 9)]

In [34]:
list(permutations([1, 2, 3]))

[(1, 2, 3), (1, 3, 2), (2, 1, 3), (2, 3, 1), (3, 1, 2), (3, 2, 1)]

In [30]:
for i in [3.14, 2.7281, 123.501]:
    pass
i

123.501

In [43]:
# note that it returns a list although the input is a tuple
sorted((3,2,1))

[1, 2, 3]

In [50]:
%%time
# (key, value) of D will be of type (tuple, list)
D = {}
S = set(range(0, 9+1))
for length in range(2, 10+1):
    for c in itertools.combinations(S, length):
        c_sorted = sorted(c)
        for p in permutations(c):
            D[p] = c_sorted

CPU times: user 6.56 s, sys: 914 ms, total: 7.47 s
Wall time: 7.5 s


Well, it took less time than I thought, even on an old laptop as Thinkpad X220 (Several seconds).

In [47]:
len(D)

9864090

In [48]:
len(D) == sum([functools.reduce(lambda x, y: x*y, range(10,10-length,-1))
     for length in range(2, 10+1)])

True

To train on sequences of diff lengths, we must **batch sequences of identical lengths together**. This poses a new difficulty...

**Rmk**. Let's **use every single one of them as our training set** and **no test set**, because in this particular task, we care less about its generalization ability. If ever later on we care that, we can include arrays of a larger variety of numbers (e.g. all floating point numbers).

Since there are

- `10*9` length-2 arrays
- `10*9*8` length-3 arrays
- ...

We can set our batch size to be `10`, a common divisor of all the numbers above.

**Rmk**. I have had no prior experience dealing with batches of different lengths of sequences. I referred to [this stackexchange post](https://datascience.stackexchange.com/questions/26366/training-an-rnn-with-examples-of-different-lengths-in-keras).

In [73]:
A = [1,2,3,4,5,]
np.random.shuffle(A)
A

[4, 1, 2, 3, 5]

In [80]:
shuffled_lengths = list(range(2, 10+1))
np.random.shuffle(shuffled_lengths)
shuffled_lengths

[6, 9, 3, 4, 5, 10, 8, 7, 2]

In [81]:
np.empty((3,2))

array([[0.0e+000, 4.9e-324],
       [1.5e-323, 2.5e-323],
       [3.5e-323, 4.4e-323]])

In [82]:
n_classes = 10

In [103]:
X = np.empty((3,3,4))
X[1:, ...] = np.arange(3*4).reshape((3,4))
X

array([[[4.63981045e-310, 0.00000000e+000, 6.93715282e-310,
         6.93715240e-310],
        [6.93715240e-310, 6.93717494e-310, 6.93717494e-310,
         6.93717494e-310],
        [6.93714615e-310, 6.93714615e-310, 6.93714615e-310,
         6.93714615e-310]],

       [[0.00000000e+000, 1.00000000e+000, 2.00000000e+000,
         3.00000000e+000],
        [4.00000000e+000, 5.00000000e+000, 6.00000000e+000,
         7.00000000e+000],
        [8.00000000e+000, 9.00000000e+000, 1.00000000e+001,
         1.10000000e+001]],

       [[0.00000000e+000, 1.00000000e+000, 2.00000000e+000,
         3.00000000e+000],
        [4.00000000e+000, 5.00000000e+000, 6.00000000e+000,
         7.00000000e+000],
        [8.00000000e+000, 9.00000000e+000, 1.00000000e+001,
         1.10000000e+001]]])

In [96]:
def one_hot(array, depth=n_classes):
    """
    array is an ndarray/list of shape (None,)
    """
    return np.eye(depth)[array, :]

In [97]:
c_sorted = np.array([0, 4, 7 ,9])
one_hot(c_sorted)

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [98]:
tf.one_hot(c_sorted, depth=n_classes)

<tf.Tensor: shape=(4, 10), dtype=float32, numpy=
array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]], dtype=float32)>

In [113]:
import math

def train_generator(batch_size=10):
    shuffled_lengths = list(range(2, 10+1))
    np.random.shuffle(shuffled_lengths)
    for length in shuffled_lengths:
        n_instances = functools.reduce(lambda x, y: x*y, range(10,10-length,-1))
        X = np.empty((n_instances, length, 1))
        Y = np.empty((n_instances, length, n_classes))
        n_permutations = math.factorial(length)
        #n_combinations = n_instances // n_permutations
        for i, c in enumerate(itertools.combinations(S, length)):
            c_sorted = np.array(sorted(c))  # shape (length,)
            c_onehot = one_hot(c_sorted)    # shape (length, n_classes)
            Y[i*n_permutations : i*n_permutations + n_permutations, ...] = c_onehot
            for j, p in enumerate(permutations(c)):
                X[i*n_permutations : i*n_permutations + j, ...] = np.array(p)[..., np.newaxis]
        # Throw one batch after another to the model
        for k in range(n_instances // batch_size):
            yield X[k*batch_size: (k+1)*batch_size], Y[k*batch_size: (k+1)*batch_size]

## Seq-to-Seq Model
This is the seq-to-seq model in which the output sequence's length equals the input sequence length. We had better used **bidirectional RNNs**.

In [52]:
seq2seq_LSTM_model = keras.models.Sequential([
    keras.layers.Bidirectional(keras.layers.LSTM(10, return_sequences=True, input_shape=[None, 1], dropout=0.2)),
    keras.layers.LSTM(10, return_sequences=True, dropout=0.2),
    #keras.layers.Bidirectional(keras.layers.LSTM(10, return_sequences=True, dropout=0.2)),
    keras.layers.TimeDistributed(keras.layers.Dense(10, activation="softmax")),
])

seq2seq_LSTM_model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")

In [ ]:
seq2seq_LSTM_model.fit(train_generator(), epochs=3)